In [1]:
data = [line.strip().split('\t') for line in open('project3_dataset2.txt', 'r')]

In [2]:
import numpy as np
import math
data = np.asarray(data)
print(data)
classlabels = data[:,-1].reshape((data.shape[0],1)).astype(int)
data = data[:,0:-1]
print(classlabels)
print(data)

[['132' '6.20' '6.47' ... '14.14' '45' '0']
 ['123' '0.05' '4.61' ... '2.78' '16' '0']
 ['128' '0.50' '3.70' ... '22.73' '28' '0']
 ...
 ['138' '4.50' '2.85' ... '24.89' '56' '1']
 ['170' '7.60' '5.50' ... '6.17' '54' '1']
 ['128' '0.00' '10.58' ... '14.66' '48' '0']]
[[0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 

In [3]:
def is_categorical(s):
    try:
        float(s)
    except ValueError:
        return True
    return False



def evaluate(actual, predicted):
    # a =TP, b=FN, c=FP, d=TN
    a = b = c = d = 0
    for i in range(len(actual)):
        if(actual[i] == 1 and predicted[i] == 1):
            a += 1
        elif(actual[i] == 1 and predicted[i] == 0):
            b += 1
        elif(actual[i] == 0 and predicted[i] == 1):
            c += 1
        elif(actual[i] == 0 and predicted[i] == 0):
            d += 1
    return a, b, c, d

In [4]:
strindex = []
for i in range(len(data[0])):
    if(is_categorical(data[0][i])):
        print(data[0][i])
        print(i)
        strindex.append(i)
for i in strindex:
    unique_vals = np.unique(data[:,i])
    mapping = list(range(len(unique_vals)))
    dictionary = dict(zip(unique_vals, mapping))
    for j in range(len(data[:,i])):
        data[j][i] = dictionary.get(data[j][i])

data = data.astype(float)
data = np.append(data,classlabels,axis=1)
print(data)

Present
4
[[1.320e+02 6.200e+00 6.470e+00 ... 1.414e+01 4.500e+01 0.000e+00]
 [1.230e+02 5.000e-02 4.610e+00 ... 2.780e+00 1.600e+01 0.000e+00]
 [1.280e+02 5.000e-01 3.700e+00 ... 2.273e+01 2.800e+01 0.000e+00]
 ...
 [1.380e+02 4.500e+00 2.850e+00 ... 2.489e+01 5.600e+01 1.000e+00]
 [1.700e+02 7.600e+00 5.500e+00 ... 6.170e+00 5.400e+01 1.000e+00]
 [1.280e+02 0.000e+00 1.058e+01 ... 1.466e+01 4.800e+01 0.000e+00]]


In [5]:
# print(data[:,4])
print(data.shape)

(462, 10)


In [6]:
def calculate_gini(left, right):
    prob_left0 = prob_left1 = prob_right0 = prob_right1 = 0
    l = len(left)
    r = len(right)
    total = l + r
    if(l != 0):
        prob_left0 = float(list(left[:,-1]).count(0)) / l
        prob_left1 = float(list(left[:,-1]).count(1)) / l
    if(r != 0):
        prob_right0 = float(list(right[:,-1]).count(0)) / r
        prob_right1 = float(list(right[:,-1]).count(1)) / r
    
    gini_left = 1.0 - ((prob_left0 ** 2) + (prob_left1 ** 2))
    gini_right = 1.0 - ((prob_right0 ** 2) + (prob_right1 ** 2))
    
    gini = float((gini_left * l) + (gini_right * r)) / total
    return gini



In [7]:
def split_data(data,j,splitval):
    left = []
    right = []
    for i in range(len(data)):
        if(data[i][j] < splitval):
            left.append(data[i])
        else:
            right.append(data[i])   
    left = np.asarray(left)
    right = np.asarray(right)
    return left, right

In [8]:
def node_to_split(data):
    min = float('inf')
    node = dict()
    random_columns = random.sample(range(0, len(data[0])-1),rand_feat)
#     features = list()
    
#     while(len(features)<rand_feat):
#         col = random.randrange(len(data[0])-1)
#         if col not in features:
#             features.append(col)

    for j in random_columns:
        for i in range(len(data)):
                left, right = split_data(data,j,data[i][j])
                gini = calculate_gini(left, right)
                if(gini < min):
                    min = gini
                    node['gini'] = min
                    node['split_id'] = i
                    node['splitval'] = data[i][j]
                    node['split_feat'] = j
                    node['left'] = left
                    node['right'] = right                                   
    return node


def node_label(left, right):
    num0s = 0
    num1s = 0
    if(len(left) != 0):
        num0s += list(left[:,-1]).count(0)
        num1s += list(left[:,-1]).count(1)
    if(len(right) != 0):
        num0s += list(right[:,-1]).count(0)
        num1s += list(right[:,-1]).count(1)
    if(num1s > num0s):
        return 1
    else:
        return 0


def is_sameclass(data):
    unique_labels = np.unique(data[:,-1])
    if(len(unique_labels) == 1):
        return True
    return False

In [9]:
def build_tree(node):
    left = node['left']
    right = node['right']
    del(node['left'])
    del(node['right'])

    if(len(left) == 0 or len(right) == 0):
        node['left'] = node['right'] = node_label(left, right)
        return node 
    #     if((left.size==0) or (right.size==0)):
#         classes = list()
#         if(left.size==0):
#             classes = right[:,-1]  
#         else:
#             classes = left[:,-1]
            
#         node['left'] = node['right'] = max(np.unique(classes), key = list(classes).count)
#         return node

    if(len(left) > 0):
        if(is_sameclass(left)):
            node['left'] = node_label(left, [])
        else:
            node['left'] = build_tree(node_to_split(left))
    if(len(right) > 0):
        if(is_sameclass(right)):
            node['right'] = node_label([], right)
        else:
            node['right'] = build_tree(node_to_split(right))        
    return node


In [10]:
def classify(node, test_point):
    
    #     if(test_point[node['split_feat']] < node['splitval']):
#         if(node['left'] == 0 or node['left'] == 1):
#             return node['left']                                 
#         else:
#             return classify(node['left'], test_point)
#     else:
#         if(node['right'] == 0 or node['right'] == 1):
#             return node['right']
#         else:
#             return classify(node['right'], test_point)
    
    if(test_point[node['split_feat']] < node['splitval']):
        if isinstance(node['left'], dict):
            return classify(node['left'], test_point)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return classify(node['right'], test_point)
        else:
            return node['right']
    


In [11]:
def split(kfoldlist,index):
    test = kfoldlist[index]
    train = np.vstack([j for i,j in enumerate(kfoldlist) if i != index])
    train = np.asarray(train)
    test = np.asarray(test)
    return train, test

In [12]:
def majority_labels(every_tree_labels):
    test_labels = []
    for i in range(len(every_tree_labels)):
        l = list(every_tree_labels[i])
        test_labels.append(max(set(l), key=l.count))
    return test_labels


def sample(data):
    return data[np.random.choice(len(data), len(data), replace=True),:]
    #     sample = list()
    #     while(len(sample)<sample_size):
    #         i = random.randrange(len(data))
    #         sample.append(data[i])
    #     sample = np.asarray(sample)
    #     return sample

    
def do_random_forest(train, test):
    every_tree_labels = []
    for i in range(ntrees):
        train_sample = sample(train)
        rootnode = node_to_split(train_sample)
        rootnode = build_tree(rootnode)
        test_labels = []
        for j in range(len(test)):
            test_labels.append(classify(rootnode,test[j]))
        every_tree_labels.append(np.asarray(test_labels))
    every_tree_labels = np.transpose(every_tree_labels)
    test_labels = []
    for i in range(len(every_tree_labels)):
        l = list(every_tree_labels[i])
        test_labels.append(max(set(l), key=l.count))
    return test_labels

In [13]:
import random

kfoldlist = np.array_split(data, 10)
accuracy = precision = recall = f_measure = 0

rand_feat = int(len(data[0]) / 3)
# rand_feat = sample_size = 0
ntrees = 10


for i in range(len(kfoldlist)):
    train, test = split(kfoldlist,i)
#     rand_feat = int(((len(train[0]))**0.5))
#     sample_size = int(len(train)/ntrees)
    test_labels = []
    test_labels = do_random_forest(train, test)
    test_labels = np.asarray(test_labels)
    a, b, c, d = evaluate(test[:,-1], test_labels)
    accuracy += float(a + d)/(a + b + c + d)
    precision += float(a)/(a + c)
    recall += float(a)/(a + b)
    f_measure += float(2 * a) / ((2 * a) + b + c)
    
    
print("Accuracy: ",accuracy*10,"%")
print("Precision: ",precision*10,"%")
print("Recall: ",recall*10,"%")
print("F-1 Measure: ",f_measure*10,"%")

Accuracy:  63.8436632747456 %
Precision:  45.71015096015096 %
Recall:  32.63346448872765 %
F-1 Measure:  37.52976833911301 %
